# <span style = 'color: #81A5FF'>  Exploratory Data Analysis and Cleaning </span>

### 1. Importing Libraries and Data

In [331]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import datetime
import requests
import random
import re
import unicodedata

from sklearn.metrics.pairwise import haversine_distances
from math import radians

# from geopy.geocoders import Nominatim
from functions.utils import *
#from functions.env_colors import *
from functions.location import *
from functions.menus import *
from functions.preprocessement import *
from functions.restaurants import *

#ignore warnings
import warnings
warnings.filterwarnings("ignore")


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [332]:
#Display all columns
pd.set_option('display.max_columns', None) 

In [333]:
data = pd.read_csv('data/alltheforkscrapes2.csv')

In [334]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [335]:
data['restaurantID'] = data['url'].str.split('-r').str[-1]
data.set_index('restaurantID', inplace=True)

In [336]:
data

,address,averagePrice,chefName,cuisine,currency,customerPhotos/0,customerPhotos/1,customerPhotos/2,customerPhotos/3,customerPhotos/4,customerPhotos/5,customerPhotos/6,customerPhotos/7,customerPhotos/8,customerPhotos/9,customerPhotos/10,customerPhotos/11,customerPhotos/12,customerPhotos/13,customerPhotos/14,customerPhotos/15,customerPhotos/16,customerPhotos/17,customerPhotos/18,customerPhotos/19,description,hasLoyaltyProgram,isBookable,latitude,location,longitude,maxPartySize,name,offer,openingHours,paymentAccepted/0,paymentAccepted/1,paymentAccepted/2,paymentAccepted/3,paymentAccepted/4,paymentAccepted/5,paymentAccepted/6,paymentAccepted/7,paymentAccepted/8,phone,photo,photos/0,photos/1,photos/2,photos/3,photos/4,photos/5,photos/6,photos/7,photos/8,photos/9,photos/10,photos/11,photos/12,photos/13,photos/14,photos/15,photos/16,photos/17,photos/18,photos/19,photos/20,photos/21,photos/22,photos/23,photos/24,photos/25,photos/26,photos/27,photos/28,photos/29,photos/30,photos/31,photos/32,photos/33,photos/34,photos/35,photos/36,photos/37,photos/38,photos/39,photos/40,photos/41,photos/42,photos/43,photos/44,photos/45,photos/46,photos/47,photos/48,photos/49,photos/50,photos/51,photos/52,photos/53,photos/54,photos/55,photos/56,photos/57,photos/58,photos/59,photos/60,photos/61,photos/62,photos/63,photos/64,photos/65,photos/66,photos/67,photos/68,photos/69,photos/70,photos/71,photos/72,photos/73,photos/74,photos/75,photos/76,photos/77,photos/78,photos/79,photos/80,photos/81,photos/82,photos/83,photos/84,photos/85,photos/86,photos/87,photos/88,photos/89,photos/90,radius,ratingValue,reviewCount,reviewList/0/ambienceRatingValue,reviewList/0/date,reviewList/0/foodRatingValue,reviewList/0/ratingValue,reviewList/0/review,reviewList/0/reviewerName,reviewList/0/serviceRatingValue,reviewList/1/ambienceRatingValue,reviewList/1/date,reviewList/1/foodRatingValue,reviewList/1/ratingValue,reviewList/1/review,reviewList/1/reviewerName,reviewList/1/serviceRatingValue,reviewList/2/ambienceRatingValue,reviewList/2/date,reviewList/2/foodRatingValue,reviewList/2/ratingValue,reviewList/2/review,reviewList/2/reviewerName,reviewList/2/serviceRatingValue,reviewList/3/ambienceRatingValue,reviewList/3/date,reviewList/3/foodRatingValue,reviewList/3/ratingValue,reviewList/3/review,reviewList/3/reviewerName,reviewList/3/serviceRatingValue,reviewList/4/ambienceRatingValue,reviewList/4/date,reviewList/4/foodRatingValue,reviewList/4/ratingValue,reviewList/4/review,reviewList/4/reviewerName,reviewList/4/serviceRatingValue,reviewList/5/ambienceRatingValue,reviewList/5/date,reviewList/5/foodRatingValue,reviewList/5/ratingValue,reviewList/5/review,reviewList/5/reviewerName,reviewList/5/serviceRatingValue,reviewList/6/ambienceRatingValue,reviewList/6/date,reviewList/6/foodRatingValue,reviewList/6/ratingValue,reviewList/6/review,reviewList/6/reviewerName,reviewList/6/serviceRatingValue,reviewList/7/ambienceRatingValue,reviewList/7/date,reviewList/7/foodRatingValue,reviewList/7/ratingValue,reviewList/7/review,reviewList/7/reviewerName,reviewList/7/serviceRatingValue,reviewList/8/ambienceRatingValue,reviewList/8/date,reviewList/8/foodRatingValue,reviewList/8/ratingValue,reviewList/8/review,reviewList/8/reviewerName,reviewList/8/serviceRatingValue,reviewList/9/ambienceRatingValue,reviewList/9/date,reviewList/9/foodRatingValue,reviewList/9/ratingValue,reviewList/9/review,reviewList/9/reviewerName,reviewList/9/serviceRatingValue,reviewList/10/ambienceRatingValue,reviewList/10/date,reviewList/10/foodRatingValue,reviewList/10/ratingValue,reviewList/10/review,reviewList/10/reviewerName,reviewList/10/serviceRatingValue,reviewList/11/ambienceRatingValue,reviewList/11/date,reviewList/11/foodRatingValue,reviewList/11/ratingValue,reviewList/11/review,reviewList/11/reviewerName,reviewList/11/serviceRatingValue,reviewList/12/ambienceRatingValue,reviewList/12/date,reviewList/12/foodRatingValue,reviewList/12/ratingValue,reviewList/12/review,reviewList/12/reviewerName,reviewList/12/serviceRat

In [337]:
data = data[~data.index.duplicated(keep='first')]

### 2. Summary Statistics

In [338]:
data.describe()

,averagePrice,latitude,longitude,maxPartySize,phone,radius,ratingValue,reviewCount,reviewList/0/ambienceRatingValue,reviewList/0/foodRatingValue,reviewList/0/ratingValue,reviewList/0/serviceRatingValue,reviewList/1/ambienceRatingValue,reviewList/1/foodRatingValue,reviewList/1/ratingValue,reviewList/1/serviceRatingValue,reviewList/2/ambienceRatingValue,reviewList/2/foodRatingValue,reviewList/2/ratingValue,reviewList/2/serviceRatingValue,reviewList/3/ambienceRatingValue,reviewList/3/foodRatingValue,reviewList/3/ratingValue,reviewList/3/serviceRatingValue,reviewList/4/ambienceRatingValue,reviewList/4/foodRatingValue,reviewList/4/ratingValue,reviewList/4/serviceRatingValue,reviewList/5/ambienceRatingValue,reviewList/5/foodRatingValue,reviewList/5/ratingValue,reviewList/5/serviceRatingValue,reviewList/6/ambienceRatingValue,reviewList/6/foodRatingValue,reviewList/6/ratingValue,reviewList/6/serviceRatingValue,reviewList/7/ambienceRatingValue,reviewList/7/foodRatingValue,reviewList/7/ratingValue,reviewList/7/serviceRatingValue,reviewList/8/ambienceRatingValue,reviewList/8/foodRatingValue,reviewList/8/ratingValue,reviewList/8/serviceRatingValue,reviewList/9/ambienceRatingValue,reviewList/9/foodRatingValue,reviewList/9/ratingValue,reviewList/9/serviceRatingValue,reviewList/10/ambienceRatingValue,reviewList/10/foodRatingValue,reviewList/10/ratingValue,reviewList/10/serviceRatingValue,reviewList/11/ambienceRatingValue,reviewList/11/foodRatingValue,reviewList/11/ratingValue,reviewList/11/serviceRatingValue,reviewList/12/ambienceRatingValue,reviewList/12/foodRatingValue,reviewList/12/ratingValue,reviewList/12/serviceRatingValue,reviewList/13/ambienceRatingValue,reviewList/13/foodRatingValue,reviewList/13/ratingValue,reviewList/13/serviceRatingValue,reviewList/14/ambienceRatingValue,reviewList/14/foodRatingValue,reviewList/14/ratingValue,reviewList/14/serviceRatingValue,reviewList/15/ambienceRatingValue,reviewList/15/foodRatingValue,reviewList/15/ratingValue,reviewList/15/serviceRatingValue,reviewList/16/ambienceRatingValue,reviewList/16/foodRatingValue,reviewList/16/ratingValue,reviewList/16/serviceRatingValue,reviewList/17/ambienceRatingValue,reviewList/17/foodRatingValue,reviewList/17/ratingValue,reviewList/17/serviceRatingValue,reviewList/18/ambienceRatingValue,reviewList/18/foodRatingValue,reviewList/18/ratingValue,reviewList/18/serviceRatingValue,reviewList/19/ambienceRatingValue,reviewList/19/foodRatingValue,reviewList/19/ratingValue,reviewList/19/serviceRatingValue
count,1632.000000,1632.000000,1632.000000,1410.000000,9.740000e+02,1.632000e+03,1592.000000,1621.000000,1583.000000,1583.000000,1583.000000,1583.000000,1520.000000,1520.000000,1520.000000,1520.000000,1473.000000,1473.000000,1473.000000,1473.000000,1422.000000,1422.000000,1422.000000,1422.000000,1391.000000,1391.000000,1391.000000,1391.000000,1358.000000,1358.000000,1358.000000,1358.000000,1330.000000,1330.000000,1330.000000,1330.000000,1298.000000,1298.000000,1298.000000,1298.000000,1256.000000,1256.000000,1256.000000,1256.000000,1224.000000,1224.000000,1224.000000,1224.000000,1191.000000,1191.000000,1191.000000,1191.000000,1150.000000,1150.000000,1150.000000,1150.000000,1125.000000,1125.000000,1125.000000,1125.000000,1100.000000,1100.000000,1100.000000,1100.000000,1072.000000,1072.000000,1072.000000,1072.000000,1052.000000,1052.000000,1052.000000,1052.000000,1034.000000,1034.000000,1034.000000,1034.000000,1004.000000,1004.000000,1004.000000,1004.000000,981.000000,981.000000,981.000000,981.000000,960.000000,960.000000,960.000000,960.000000
mean,26.087010,40.169333,-8.305869,51.445390,3.395265e+11,7.245232e+13,8.983040,560.998766,9.080227,9.163613,9.147505,9.182565,9.035526,9.185526,9.150658,9.196053,9.093007,9.170401,9.168364,9.239647,9.059072,9.194093,9.159634,9.191280,9.079799,9.181884,9.167505,9.226456,9.185567,9.300442,9.272091,9.301915,9.094737,9.186466,9.173684,9.227068,9.149461,9.212635,9.206086,9.249615,9.136943,9.248408,9.222532,9.256369,9.173203,9.292484,9.262663

In [339]:
data.describe(include='object')

,address,chefName,cuisine,currency,customerPhotos/0,customerPhotos/1,customerPhotos/2,customerPhotos/3,customerPhotos/4,customerPhotos/5,customerPhotos/6,customerPhotos/7,customerPhotos/8,customerPhotos/9,customerPhotos/10,customerPhotos/11,customerPhotos/12,customerPhotos/13,customerPhotos/14,customerPhotos/15,customerPhotos/16,customerPhotos/17,customerPhotos/18,customerPhotos/19,description,location,name,offer,openingHours,paymentAccepted/0,paymentAccepted/1,paymentAccepted/2,paymentAccepted/3,paymentAccepted/4,paymentAccepted/5,paymentAccepted/6,paymentAccepted/7,paymentAccepted/8,photo,photos/0,photos/1,photos/2,photos/3,photos/4,photos/5,photos/6,photos/7,photos/8,photos/9,photos/10,photos/11,photos/12,photos/13,photos/14,photos/15,photos/16,photos/17,photos/18,photos/19,photos/20,photos/21,photos/22,photos/23,photos/24,photos/25,photos/26,photos/27,photos/28,photos/29,photos/30,photos/31,photos/32,photos/33,photos/34,photos/35,photos/36,photos/37,photos/38,photos/39,photos/40,photos/41,photos/42,photos/43,photos/44,photos/45,photos/46,photos/47,photos/48,photos/49,photos/50,photos/51,photos/52,photos/53,photos/54,photos/55,photos/56,photos/57,photos/58,photos/59,photos/60,photos/61,photos/62,photos/63,photos/64,photos/65,photos/66,photos/67,photos/68,photos/69,photos/70,photos/71,photos/72,photos/73,photos/74,photos/75,photos/76,photos/77,photos/78,photos/79,photos/80,photos/81,photos/82,photos/83,photos/84,photos/85,photos/86,photos/87,photos/88,photos/89,photos/90,reviewList/0/date,reviewList/0/review,reviewList/0/reviewerName,reviewList/1/date,reviewList/1/review,reviewList/1/reviewerName,reviewList/2/date,reviewList/2/review,reviewList/2/reviewerName,reviewList/3/date,reviewList/3/review,reviewList/3/reviewerName,reviewList/4/date,reviewList/4/review,reviewList/4/reviewerName,reviewList/5/date,reviewList/5/review,reviewList/5/reviewerName,reviewList/6/date,reviewList/6/review,reviewList/6/reviewerName,reviewList/7/date,reviewList/7/review,reviewList/7/reviewerName,reviewList/8/date,reviewList/8/review,reviewList/8/reviewerName,reviewList/9/date,reviewList/9/review,reviewList/9/reviewerName,reviewList/10/date,reviewList/10/review,reviewList/10/reviewerName,reviewList/11/date,reviewList/11/review,reviewList/11/reviewerName,reviewList/12/date,reviewList/12/review,reviewList/12/reviewerName,reviewList/13/date,reviewList/13/review,reviewList/13/reviewerName,reviewList/14/date,reviewList/14/review,reviewList/14/reviewerName,reviewList/15/date,reviewList/15/review,reviewList/15/reviewerName,reviewList/16/date,reviewList/16/review,reviewList/16/reviewerName,reviewList/17/date,reviewList/17/review,reviewList/17/reviewerName,reviewList/18/date,reviewList/18/review,reviewList/18/reviewerName,reviewList/19/date,reviewList/19/review,reviewList/19/reviewerName,style,tags/0,tags/1,tags/2,tags/3,tags/4,tags/5,tags/6,tags/7,url,paymentAccepted/9,paymentAccepted/10,photos/91,photos/92,photos/93,photos/94,photos/95,photos/96,photos/97,photos/98,photos/99,photos/100,photos/101,photos/102,photos/103,photos/104,photos/105
count,1632,658,1630,1632,1540,1493,1452,1416,1376,1346,1311,1282,1242,1215,1184,1155,1131,1110,1082,1066,1044,1019,996,973,25,1522,1632,524,1605,1582,1394,1189,713,396,224,88,13,7,1632,1632,1631,1622,1584,1522,1448,1350,1265,1181,1081,953,895,815,753,684,621,594,549,500,457,419,390,359,321,296,269,248,226,200,182,156,148,138,131,122,113,99,92,84,81,70,64,61,55,51,45,44,44,44,41,40,38,38,37,35,32,31,29,27,26,25,25,24,23,20,20,20,19,19,16,14,13,11,10,9,8,8,7,6,5,5,5,5,5,5,5,5,5,4,4,3,1583,1583,1583,1520,1520,1520,1473,1473,1473,1422,1421,1422,1391,1391,1391,1358,1358,1358,1330,1330,1330,1298,1298,1298,1256,1256,1256,1224,1223,1224,1191,1191,1191,1150,1150,1150,1125,1125,1125,1100,1100,1100,1072,1072,1072,1052,1052,1052,1034,1034,1034,1004,1004,1004,981,981,981,960,960,960,1571,1632,450,162,74,45,23,7,2,1632,2,1,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1
unique,1618,611,56,5,1540,1493,1452,1416,1376,1346,1311,1282,1242,1215,1184,1155,1

### 3. Extracting Locations

In [340]:
locations = data[['name','address', 'location', 'longitude', 'latitude']].copy()

In [341]:
#Finds last occurence of the address to determine the city
locations['location2'] = locations['address'].map(lambda x: str(x).split(',')[-1])

#Finds the first occurence within "location" to find city -> this was how we eextracted the data
locations['location3'] = locations['location'].map(lambda x: str(x).split(',')[0])

In [342]:
#Finding indexes from observations not based in Portugal and removing them from the original data and creating a new dataframe with only portuguese location info
foreign_indexes = list((locations[(locations['location'].isna())]).index)
data.drop(foreign_indexes, inplace=True, axis=0)
pt_locations = locations[locations['location'].isna() == False].copy()

In [343]:
#Pre processing address to further find latitude and longitude coordinates
pt_locations['address'] = pt_locations['address'].apply(preprocess_address)

In [344]:
# pt_locations[['latitude', 'longitude']] = pt_locations['address'].apply(lambda x: pd.Series(find_coordinates(x)))

In [345]:
# pt_locations.to_csv('data/portuguese_locations.csv')

In [346]:
#Step intermédio enquanto o notebook nao estiver finalizado
pt_locations2 = pd.read_csv('data/portuguese_locations.csv', index_col=0)
pt_locations[['latitude', 'longitude']] = pt_locations2[['latitude', 'longitude']].copy()

In [347]:
data[['address', 'latitude', 'longitude', 'location', 'city']] = pt_locations[['address', 'latitude', 'longitude', 'location2', 'location3']].copy()

### 4. Handling Missing Values and Standardizing Values

In [348]:
null_columns = data.isnull().any()
null_columns = null_columns[null_columns].index
#list(null_columns)

#### Chef Name

In [349]:
data['chefName'] = np.where(data['chefName'].isnull(), 'Not Applicable', data['chefName'])

In [350]:
data['chefes'] = data['chefName'].apply(get_chef_name)
data['chefName1'] = data['chefes'].apply(lambda x: preprocess_chefs(0, x))
data['chefName2'] = data['chefes'].apply(lambda x: preprocess_chefs(1, x))
data['chefName3'] = data['chefes'].apply(lambda x: preprocess_chefs(2, x))

#### Cuisine

In [351]:
data['cuisine'].unique()

array(['International food', 'Japanese', 'Indian', 'Portuguese',
       'Italian', 'Pizzeria', 'International', 'Mediterranean', 'Fusion',
       'Nepalese', 'European', 'Seafood', 'Vegan cuisine',
       'Traditional cuisine', 'Steakhouse', 'Regional', 'Greek',
       'Vegetarian', 'Varied', 'Grilled', 'Thai', 'Mexican', 'Local',
       'Asian', 'French', 'South American', 'Pub grub', 'Brazilian',
       'Venezuelan', 'Peruvian', 'Meat Cuisine', 'Korean', 'American',
       'Toscano', 'Spanish', 'African', 'Syrian', 'Iranian', 'Lebanese',
       'Cantonese', 'Tibetan', 'Vietnamese', 'Argentinian', 'Pugliese',
       'Chinese'], dtype=object)

In [352]:
data['cuisine'] = data['cuisine'].replace({'International food': 'International', 
                                           'Toscano': 'Italian', 
                                           'Pugliese': 'Italian', 
                                           'Traditional cuisine': 'Portuguese', 
                                           'Regional': 'Portuguese', 
                                           'Local': 'Portuguese', 
                                           'Cantonese': 'Chinese'})

In [353]:
data['cuisine_standard'] = data['cuisine'].apply(lambda x: standardize_text(x))

#### Phone Number

In [354]:
data['phone'] = data['phone'].astype(str)
data['phone'] = data['phone'].apply(lambda x: x[:-2] if x != 'nan' else 'Not Available')
data['phone'] = data['phone'].apply(lambda x: x[3:12] if len(x) == 12 and x.startswith('351') and x != 'Not Available' else 'Not Available')

#### Dealing with Schedule Format

In [355]:
data['openingHours'].fillna('Not Available', inplace=True)

In [356]:
#Clearing the openingHours column
data['schedule'] = data['openingHours'].apply(lambda x: clean_openinghours(x))

#### Generating Promotions

In [357]:
data['promotions'] = data['schedule'].apply(lambda x: promotion_generator(x, 3, 0.5))

#### Standardizing Locations

In [358]:
data['location_standard'] = data['location'].apply(lambda x: standardize_location(x))
data['city_standard'] = data['city'].apply(lambda x: standardize_location(x))

#### Standardizing Style

In [359]:
data['style'].fillna('Not Available', inplace=True)

In [360]:
data['style'].unique()

array(['Bachelor party', 'After work', 'Birthday', 'Good for ceremonies',
       'All you can eat buffet', 'Good for families',
       'Contemporary cuisine', 'Author', 'Good for groups',
       'Central location', 'Bistro', 'Not Available', 'Fine Dining',
       'Cosy', 'Live music', 'Creative', 'Brunch', 'Lunch', 'Traditional',
       'Bistronomic', 'Good for a business lunch', 'Terrace',
       'Homemade cuisine', 'Brasserie', 'Garden', 'Oceanfront',
       'Great view', 'Café', 'Trendy', 'Romantic', 'Restaurant hotel',
       'Communion', 'Breakfast', 'Winter terrace', 'Nightlife',
       'With friends', 'Street Food', 'Organic', 'Healthy.old', 'Ethnic',
       'From market', 'Design', 'Wine bar'], dtype=object)

In [361]:
data['style_standard'] = data['style'].apply(lambda x: standardize_text(x))

### 6. Exploratory Data Analysis (through Sections)

#### 6.1. Photos

In [362]:
photos = ['photo', 'customerPhotos/0','customerPhotos/1', 'customerPhotos/2', 'customerPhotos/3', 'customerPhotos/4', 'customerPhotos/5', 'customerPhotos/6',
 'customerPhotos/7', 'customerPhotos/8', 'customerPhotos/9', 'customerPhotos/10', 'customerPhotos/11', 'customerPhotos/12', 'customerPhotos/13',
 'customerPhotos/14', 'customerPhotos/15', 'customerPhotos/16', 'customerPhotos/17', 'customerPhotos/18', 'customerPhotos/19', 'photos/1',
 'photos/2', 'photos/3', 'photos/4', 'photos/5', 'photos/6', 'photos/7','photos/8','photos/9','photos/10', 'photos/11', 'photos/12', 'photos/13', 'photos/14',
 'photos/15','photos/16', 'photos/17', 'photos/18', 'photos/19', 'photos/20', 'photos/21', 'photos/22', 'photos/23', 'photos/24', 'photos/25', 'photos/26', 'photos/27',
 'photos/28', 'photos/29', 'photos/30', 'photos/31', 'photos/32', 'photos/33', 'photos/34', 'photos/35', 'photos/36', 'photos/37', 'photos/38', 'photos/39', 'photos/40',
 'photos/41', 'photos/42', 'photos/43', 'photos/44', 'photos/45', 'photos/46', 'photos/47', 'photos/48', 'photos/49', 'photos/50', 'photos/51', 'photos/52','photos/53',
 'photos/54', 'photos/55', 'photos/56', 'photos/57', 'photos/58', 'photos/59', 'photos/60', 'photos/61', 'photos/62', 'photos/63', 'photos/64', 'photos/65', 'photos/66',
 'photos/67','photos/68','photos/69', 'photos/70', 'photos/71', 'photos/72', 'photos/73', 'photos/74', 'photos/75', 'photos/76', 'photos/77', 'photos/78', 'photos/79',
 'photos/80', 'photos/81', 'photos/82', 'photos/83', 'photos/84', 'photos/85', 'photos/86', 'photos/87', 'photos/88', 'photos/89', 'photos/90',  'photos/91', 'photos/92',
 'photos/93', 'photos/94', 'photos/95', 'photos/96', 'photos/97', 'photos/98', 'photos/99', 'photos/100', 'photos/101', 'photos/102', 'photos/103', 'photos/104', 'photos/105']

In [363]:
df_photos = data[photos].copy()

In [364]:
df_photos['photos/1'].unique()

array(['https://res.cloudinary.com/tf-lab/image/upload/f_auto,q_auto,w_664,h_374/restaurant/0fb87f67-79d8-4ad5-bb06-6e9d668400cd/8d85bcd4-f6d9-47a3-967d-6be6f376d251.jpg',
       'https://res.cloudinary.com/tf-lab/image/upload/f_auto,q_auto,w_664,h_374/restaurant/cf797b76-9f49-4e2d-bd25-89b05d312ff3/5ffd061d-3b83-43d9-aa13-7bf39afecfd7.jpg',
       'https://res.cloudinary.com/tf-lab/image/upload/f_auto,q_auto,w_664,h_374/restaurant/25442534-4b23-40e5-91a5-2eb2da7b7a92/86b9de08-b5f8-4c6f-97c9-f8ee4dd0ad4f.jpg',
       ...,
       'https://res.cloudinary.com/tf-lab/image/upload/f_auto,q_auto,w_664,h_374/restaurant/7252fa4a-63a4-43bb-b4d5-83e3c48db045/63238949-2ad2-47e7-a812-8c4a215fbef9.jpg',
       'https://res.cloudinary.com/tf-lab/image/upload/f_auto,q_auto,w_664,h_374/restaurant/97249dc8-13a3-4473-bd9b-c9de1ada5f54/8bbb15f6-03df-4f59-8daa-73643c0b737d.jpg',
       'https://res.cloudinary.com/tf-lab/image/upload/f_auto,q_auto,w_664,h_374/restaurant/ac8c8d50-6309-4eae-bf6a-bb59fa7b9646

#### 6.2. Reviews

In [365]:
restaurant_reviews = data[['name', 'reviewList/0/ambienceRatingValue','reviewList/0/date', 'reviewList/0/foodRatingValue', 'reviewList/0/ratingValue', 'reviewList/0/review',
 'reviewList/0/reviewerName', 'reviewList/0/serviceRatingValue', 'reviewList/1/ambienceRatingValue', 'reviewList/1/date', 'reviewList/1/foodRatingValue', 'reviewList/1/ratingValue',
 'reviewList/1/review', 'reviewList/1/reviewerName', 'reviewList/1/serviceRatingValue', 'reviewList/2/ambienceRatingValue', 'reviewList/2/date', 'reviewList/2/foodRatingValue',
 'reviewList/2/ratingValue', 'reviewList/2/review', 'reviewList/2/reviewerName', 'reviewList/2/serviceRatingValue', 'reviewList/3/ambienceRatingValue','reviewList/3/date',
 'reviewList/3/foodRatingValue', 'reviewList/3/ratingValue', 'reviewList/3/review', 'reviewList/3/reviewerName', 'reviewList/3/serviceRatingValue', 'reviewList/4/ambienceRatingValue',
 'reviewList/4/date','reviewList/4/foodRatingValue', 'reviewList/4/ratingValue', 'reviewList/4/review', 'reviewList/4/reviewerName', 'reviewList/4/serviceRatingValue',
 'reviewList/5/ambienceRatingValue', 'reviewList/5/date', 'reviewList/5/foodRatingValue', 'reviewList/5/ratingValue', 'reviewList/5/review', 'reviewList/5/reviewerName',
 'reviewList/5/serviceRatingValue', 'reviewList/6/ambienceRatingValue', 'reviewList/6/date', 'reviewList/6/foodRatingValue', 'reviewList/6/ratingValue', 'reviewList/6/review',
 'reviewList/6/reviewerName', 'reviewList/6/serviceRatingValue', 'reviewList/7/ambienceRatingValue', 'reviewList/7/date', 'reviewList/7/foodRatingValue', 'reviewList/7/ratingValue',
 'reviewList/7/review', 'reviewList/7/reviewerName', 'reviewList/7/serviceRatingValue', 'reviewList/8/ambienceRatingValue', 'reviewList/8/date', 'reviewList/8/foodRatingValue',
 'reviewList/8/ratingValue', 'reviewList/8/review', 'reviewList/8/reviewerName', 'reviewList/8/serviceRatingValue', 'reviewList/9/ambienceRatingValue', 'reviewList/9/date',
 'reviewList/9/foodRatingValue', 'reviewList/9/ratingValue', 'reviewList/9/review', 'reviewList/9/reviewerName', 'reviewList/9/serviceRatingValue', 'reviewList/10/ambienceRatingValue',
 'reviewList/10/date', 'reviewList/10/foodRatingValue', 'reviewList/10/ratingValue', 'reviewList/10/review', 'reviewList/10/reviewerName', 'reviewList/10/serviceRatingValue',
 'reviewList/11/ambienceRatingValue', 'reviewList/11/date', 'reviewList/11/foodRatingValue', 'reviewList/11/ratingValue', 'reviewList/11/review', 'reviewList/11/reviewerName',
 'reviewList/11/serviceRatingValue', 'reviewList/12/ambienceRatingValue', 'reviewList/12/date', 'reviewList/12/foodRatingValue', 'reviewList/12/ratingValue', 'reviewList/12/review',
 'reviewList/12/reviewerName', 'reviewList/12/serviceRatingValue', 'reviewList/13/ambienceRatingValue', 'reviewList/13/date', 'reviewList/13/foodRatingValue', 'reviewList/13/ratingValue',
 'reviewList/13/review', 'reviewList/13/reviewerName', 'reviewList/13/serviceRatingValue', 'reviewList/14/ambienceRatingValue', 'reviewList/14/date',
 'reviewList/14/foodRatingValue', 'reviewList/14/ratingValue', 'reviewList/14/review', 'reviewList/14/reviewerName', 'reviewList/14/serviceRatingValue', 'reviewList/15/ambienceRatingValue',
 'reviewList/15/date', 'reviewList/15/foodRatingValue', 'reviewList/15/ratingValue', 'reviewList/15/review', 'reviewList/15/reviewerName', 'reviewList/15/serviceRatingValue',
 'reviewList/16/ambienceRatingValue', 'reviewList/16/date', 'reviewList/16/foodRatingValue', 'reviewList/16/ratingValue', 'reviewList/16/review', 'reviewList/16/reviewerName',
 'reviewList/16/serviceRatingValue', 'reviewList/17/ambienceRatingValue', 'reviewList/17/date', 'reviewList/17/foodRatingValue', 'reviewList/17/ratingValue', 'reviewList/17/review',
 'reviewList/17/reviewerName', 'reviewList/17/serviceRatingValue', 'reviewList/18/ambienceRatingValue', 'reviewList/18/date', 'reviewList/18/foodRatingValue', 'reviewList/18/ratingValue',
 'reviewList/18/review', 'reviewList/18/reviewerName', 'reviewList/18/serviceRatingValue', 'reviewList/19/ambienceRatingValue', 'reviewList/19/date', 'reviewList/19/foodRatingValue',
 'reviewList/19/ratingValue', 'reviewList/19/review', 'reviewList/19/reviewerName', 'reviewList/19/serviceRatingValue']].copy()

In [366]:
#restaurant_reviews.reset_index(inplace=True)
restaurant_reviews

,restaurantID,name,reviewList/0/ambienceRatingValue,reviewList/0/date,reviewList/0/foodRatingValue,reviewList/0/ratingValue,reviewList/0/review,reviewList/0/reviewerName,reviewList/0/serviceRatingValue,reviewList/1/ambienceRatingValue,reviewList/1/date,reviewList/1/foodRatingValue,reviewList/1/ratingValue,reviewList/1/review,reviewList/1/reviewerName,reviewList/1/serviceRatingValue,reviewList/2/ambienceRatingValue,reviewList/2/date,reviewList/2/foodRatingValue,reviewList/2/ratingValue,reviewList/2/review,reviewList/2/reviewerName,reviewList/2/serviceRatingValue,reviewList/3/ambienceRatingValue,reviewList/3/date,reviewList/3/foodRatingValue,reviewList/3/ratingValue,reviewList/3/review,reviewList/3/reviewerName,reviewList/3/serviceRatingValue,reviewList/4/ambienceRatingValue,reviewList/4/date,reviewList/4/foodRatingValue,reviewList/4/ratingValue,reviewList/4/review,reviewList/4/reviewerName,reviewList/4/serviceRatingValue,reviewList/5/ambienceRatingValue,reviewList/5/date,reviewList/5/foodRatingValue,reviewList/5/ratingValue,reviewList/5/review,reviewList/5/reviewerName,reviewList/5/serviceRatingValue,reviewList/6/ambienceRatingValue,reviewList/6/date,reviewList/6/foodRatingValue,reviewList/6/ratingValue,reviewList/6/review,reviewList/6/reviewerName,reviewList/6/serviceRatingValue,reviewList/7/ambienceRatingValue,reviewList/7/date,reviewList/7/foodRatingValue,reviewList/7/ratingValue,reviewList/7/review,reviewList/7/reviewerName,reviewList/7/serviceRatingValue,reviewList/8/ambienceRatingValue,reviewList/8/date,reviewList/8/foodRatingValue,reviewList/8/ratingValue,reviewList/8/review,reviewList/8/reviewerName,reviewList/8/serviceRatingValue,reviewList/9/ambienceRatingValue,reviewList/9/date,reviewList/9/foodRatingValue,reviewList/9/ratingValue,reviewList/9/review,reviewList/9/reviewerName,reviewList/9/serviceRatingValue,reviewList/10/ambienceRatingValue,reviewList/10/date,reviewList/10/foodRatingValue,reviewList/10/ratingValue,reviewList/10/review,reviewList/10/reviewerName,reviewList/10/serviceRatingValue,reviewList/11/ambienceRatingValue,reviewList/11/date,reviewList/11/foodRatingValue,reviewList/11/ratingValue,reviewList/11/review,reviewList/11/reviewerName,reviewList/11/serviceRatingValue,reviewList/12/ambienceRatingValue,reviewList/12/date,reviewList/12/foodRatingValue,reviewList/12/ratingValue,reviewList/12/review,reviewList/12/reviewerName,reviewList/12/serviceRatingValue,reviewList/13/ambienceRatingValue,reviewList/13/date,reviewList/13/foodRatingValue,reviewList/13/ratingValue,reviewList/13/review,reviewList/13/reviewerName,reviewList/13/serviceRatingValue,reviewList/14/ambienceRatingValue,reviewList/14/date,reviewList/14/foodRatingValue,reviewList/14/ratingValue,reviewList/14/review,reviewList/14/reviewerName,reviewList/14/serviceRatingValue,reviewList/15/ambienceRatingValue,reviewList/15/date,reviewList/15/foodRatingValue,reviewList/15/ratingValue,reviewList/15/review,reviewList/15/reviewerName,reviewList/15/serviceRatingValue,reviewList/16/ambienceRatingValue,reviewList/16/date,reviewList/16/foodRatingValue,reviewList/16/ratingValue,reviewList/16/review,reviewList/16/reviewerName,reviewList/16/serviceRatingValue,reviewList/17/ambienceRatingValue,reviewList/17/date,reviewList/17/foodRatingValue,reviewList/17/ratingValue,reviewList/17/review,reviewList/17/reviewerName,reviewList/17/serviceRatingValue,reviewList/18/ambienceRatingValue,reviewList/18/date,reviewList/18/foodRatingValue,reviewList/18/ratingValue,reviewList/18/review,reviewList/18/reviewerName,reviewList/18/serviceRatingValue,reviewList/19/ambienceRatingValue,reviewList/19/date,reviewList/19/foodRatingValue,reviewList/19/ratingValue,reviewList/19/review,reviewList/19/reviewerName,reviewList/19/serviceRatingValue
0,730060,Invictus,10.0,2023-08-08T19:30:00.000Z,10.0,10.0,"Espaço agradável, simpatia e interacção por pa...",Rui Jorge M.,10.0,10.0,2022-06-05T20:00:00.000Z,10.0,10.0,Our waitress Ana was fantastic. She served our...,Jim S.,10.0,NaN,NaN,NaN,NaN,NaN,N

In [367]:
#REVER PORQUE ACHO QUE NÃO ESTÁ 100% CORRETO

# Number of columns to extract
num_columns = 20  # Adjust this as needed

# Create a list of the column lists
column_lists = []

for i in range(num_columns):
    columns_to_extract = [f'reviewList/{i}/date', f'reviewList/{i}/reviewerName'] + ['restaurantID'] + [f'reviewList/{i}/ambienceRatingValue', f'reviewList/{i}/foodRatingValue', f'reviewList/{i}/serviceRatingValue', f'reviewList/{i}/review', f'reviewList/{i}/ratingValue']
    column_lists.append(columns_to_extract)

list = []

for index, row in restaurant_reviews.iterrows():
    for columns in column_lists:
        # Create a list to store the extracted values
        values = []

        # Flag to check if a nan value is encountered
        nan_encountered = False

        # Check if each column exists in the row before extracting
        for column in columns:
            if column in row.index:
                value = row[column]
                values.append(value)
            else:
                # Handle the case where the column is not found (e.g., insert a placeholder)
                values.append(None)  # You can choose an appropriate placeholder
                nan_encountered = True

            if pd.isna(value):
                nan_encountered = True

        # If a nan value is encountered, break out of the loop
        if nan_encountered:
            break

        list.append(values)

# Create a DataFrame from the list of rows
df = pd.DataFrame(list, columns=column_lists[0])  # You can use the columns from the first list

# Print the DataFrame
df

,reviewList/0/date,reviewList/0/reviewerName,restaurantID,reviewList/0/ambienceRatingValue,reviewList/0/foodRatingValue,reviewList/0/serviceRatingValue,reviewList/0/review,reviewList/0/ratingValue
0,2023-08-08T19:30:00.000Z,Rui Jorge M.,730060,10.0,10.0,10.0,"Espaço agradável, simpatia e interacção por pa...",10.0
1,2022-06-05T20:00:00.000Z,Jim S.,730060,10.0,10.0,10.0,Our waitress Ana was fantastic. She served our...,10.0
2,2023-09-23T20:30:00.000Z,Lara P.,805058,8.0,2.0,2.0,ninguém gostou,3.5
3,2023-09-21T19:30:00.000Z,James W.,805058,6.0,6.0,8.0,"The restaurant was absolutely fine, but the re...",6.5
4,2023-09-21T19:00:00.000Z,James S.,805058,10.0,10.0,10.0,We were here two nights in a row - meat porti...,10.0
...,...,...,...,...,...,...,...,...
22733,2023-07-03T18:30:00.000Z,Helena s.,612699,10.0,10.0,10.0,É sempre bom voltar ao Me.at. \nAdoramos a sua...,10.0
22734,2023-06-24T19:30:00.000Z,Cláudia S.,612699,8.0,10.0,10.0,"Espaço muito bem decorado, ambiente agradável,...",9.5
22735,2023-06-12T18:00:00.000Z,Pedro C.,612699,8.0,8.0,8.0,"Muito agradável, embora ainda calmo, pois foi ...",8.0
22736,2023-08-06T18:30:00.000Z,Rodrigo A.,802161,10.0,10.0,10.0,Comida muito boa e funcionários muito simpátic...,10.0


In [368]:
reviews = ['reviewList/0/ambienceRatingValue','reviewList/0/date', 'reviewList/0/foodRatingValue', 'reviewList/0/ratingValue', 'reviewList/0/review',
 'reviewList/0/reviewerName', 'reviewList/0/serviceRatingValue', 'reviewList/1/ambienceRatingValue', 'reviewList/1/date', 'reviewList/1/foodRatingValue', 'reviewList/1/ratingValue',
 'reviewList/1/review', 'reviewList/1/reviewerName', 'reviewList/1/serviceRatingValue', 'reviewList/2/ambienceRatingValue', 'reviewList/2/date', 'reviewList/2/foodRatingValue',
 'reviewList/2/ratingValue', 'reviewList/2/review', 'reviewList/2/reviewerName', 'reviewList/2/serviceRatingValue', 'reviewList/3/ambienceRatingValue','reviewList/3/date',
 'reviewList/3/foodRatingValue', 'reviewList/3/ratingValue', 'reviewList/3/review', 'reviewList/3/reviewerName', 'reviewList/3/serviceRatingValue', 'reviewList/4/ambienceRatingValue',
 'reviewList/4/date','reviewList/4/foodRatingValue', 'reviewList/4/ratingValue', 'reviewList/4/review', 'reviewList/4/reviewerName', 'reviewList/4/serviceRatingValue',
 'reviewList/5/ambienceRatingValue', 'reviewList/5/date', 'reviewList/5/foodRatingValue', 'reviewList/5/ratingValue', 'reviewList/5/review', 'reviewList/5/reviewerName',
 'reviewList/5/serviceRatingValue', 'reviewList/6/ambienceRatingValue', 'reviewList/6/date', 'reviewList/6/foodRatingValue', 'reviewList/6/ratingValue', 'reviewList/6/review',
 'reviewList/6/reviewerName', 'reviewList/6/serviceRatingValue', 'reviewList/7/ambienceRatingValue', 'reviewList/7/date', 'reviewList/7/foodRatingValue', 'reviewList/7/ratingValue',
 'reviewList/7/review', 'reviewList/7/reviewerName', 'reviewList/7/serviceRatingValue', 'reviewList/8/ambienceRatingValue', 'reviewList/8/date', 'reviewList/8/foodRatingValue',
 'reviewList/8/ratingValue', 'reviewList/8/review', 'reviewList/8/reviewerName', 'reviewList/8/serviceRatingValue', 'reviewList/9/ambienceRatingValue', 'reviewList/9/date',
 'reviewList/9/foodRatingValue', 'reviewList/9/ratingValue', 'reviewList/9/review', 'reviewList/9/reviewerName', 'reviewList/9/serviceRatingValue', 'reviewList/10/ambienceRatingValue',
 'reviewList/10/date', 'reviewList/10/foodRatingValue', 'reviewList/10/ratingValue', 'reviewList/10/review', 'reviewList/10/reviewerName', 'reviewList/10/serviceRatingValue',
 'reviewList/11/ambienceRatingValue', 'reviewList/11/date', 'reviewList/11/foodRatingValue', 'reviewList/11/ratingValue', 'reviewList/11/review', 'reviewList/11/reviewerName',
 'reviewList/11/serviceRatingValue', 'reviewList/12/ambienceRatingValue', 'reviewList/12/date', 'reviewList/12/foodRatingValue', 'reviewList/12/ratingValue', 'reviewList/12/review',
 'reviewList/12/reviewerName', 'reviewList/12/serviceRatingValue', 'reviewList/13/ambienceRatingValue', 'reviewList/13/date', 'reviewList/13/foodRatingValue', 'reviewList/13/ratingValue',
 'reviewList/13/review', 'reviewList/13/reviewerName', 'reviewList/13/serviceRatingValue', 'reviewList/14/ambienceRatingValue', 'reviewList/14/date',
 'reviewList/14/foodRatingValue', 'reviewList/14/ratingValue', 'reviewList/14/review', 'reviewList/14/reviewerName', 'reviewList/14/serviceRatingValue', 'reviewList/15/ambienceRatingValue',
 'reviewList/15/date', 'reviewList/15/foodRatingValue', 'reviewList/15/ratingValue', 'reviewList/15/review', 'reviewList/15/reviewerName', 'reviewList/15/serviceRatingValue',
 'reviewList/16/ambienceRatingValue', 'reviewList/16/date', 'reviewList/16/foodRatingValue', 'reviewList/16/ratingValue', 'reviewList/16/review', 'reviewList/16/reviewerName',
 'reviewList/16/serviceRatingValue', 'reviewList/17/ambienceRatingValue', 'reviewList/17/date', 'reviewList/17/foodRatingValue', 'reviewList/17/ratingValue', 'reviewList/17/review',
 'reviewList/17/reviewerName', 'reviewList/17/serviceRatingValue', 'reviewList/18/ambienceRatingValue', 'reviewList/18/date', 'reviewList/18/foodRatingValue', 'reviewList/18/ratingValue',
 'reviewList/18/review', 'reviewList/18/reviewerName', 'reviewList/18/serviceRatingValue', 'reviewList/19/ambienceRatingValue', 'reviewList/19/date', 'reviewList/19/foodRatingValue',
 'reviewList/19/ratingValue', 'reviewList/19/review', 'reviewList/19/reviewerName', 'reviewList/19/serviceRatingValue']

In [369]:
df_reviews = data[reviews].copy()

In [370]:
ambience = []
food = []
service = []
for col in df_reviews.columns:
    if 'ambienceRatingValue' in col:
        ambience.append(col)
    elif 'foodRatingValue' in col:
        food.append(col)
    elif 'serviceRatingValue' in col:
        service.append(col)

In [371]:
#Creating three new columns regarding ratings per category using the mean values of the user-based ratings
df_reviews['ambienceRatingSummary'] = df_reviews[ambience].mean(axis=1)
df_reviews['foodRatingSummary'] = df_reviews[food].mean(axis=1)
df_reviews['serviceRatingSummary'] = df_reviews[service].mean(axis=1)

In [372]:
data[['ambienceRatingSummary', 'foodRatingSummary', 'serviceRatingSummary']] = df_reviews[['ambienceRatingSummary', 'foodRatingSummary', 'serviceRatingSummary']].copy()

#### 6.3. Payment Methods

In [373]:
payments = ['paymentAccepted/0','paymentAccepted/1', 'paymentAccepted/2', 'paymentAccepted/3', 'paymentAccepted/4',
 'paymentAccepted/5', 'paymentAccepted/6', 'paymentAccepted/7', 'paymentAccepted/8', 'paymentAccepted/9', 'paymentAccepted/10']

In [374]:
df_payments = data[payments].copy()

In [375]:
df_payments.fillna(0, inplace=True)

In [376]:
df_payments['paymentAcceptedSummary'] = df_payments.apply(lambda row: [row[col] for col in df_payments.columns if row[col] != 0], axis=1)

In [377]:
data['paymentAcceptedSummary'] = df_payments['paymentAcceptedSummary'].copy()

In [378]:
data_exploded_pay = data['paymentAcceptedSummary'].explode()
payment_counts = data_exploded_pay.value_counts()

In [379]:
#Replace 'Cash Only' with 'Cash'
data['paymentAcceptedSummary'] = [x if 'Cash Only' not in x else [item.replace('Cash Only', 'Cash') for item in x] for x in data['paymentAcceptedSummary']]

#Replace 'Elo', 'Cabal Credit Card', 'Cabal', 'MobilePay', 'Rede Shop', 'Clave', 'EC Card' with 'MBWay'
data['paymentAcceptedSummary'] = [ [item if item not in ['Elo', 'Cabal Credit Card', 'Cabal Debit Card', 'MobilePay', 'Rede Shop',
                                                         'Clave Debit Card', 'Clave Credit Card', 'EC card'] else 'MBWay' for item in x]
                                  for x in data['paymentAcceptedSummary']]

# Eleminate duplicates from the list
data['paymentAcceptedSummary'] = [list(set(x)) for x in data['paymentAcceptedSummary']]

TypeError: 'list' object is not callable

#### 6.4. Tags

In [ ]:
df_tags = data[['tags/0','tags/1', 'tags/2', 'tags/3', 'tags/4', 'tags/5', 'tags/6', 'tags/7',]].copy()
df_tags

,tags/0,tags/1,tags/2,tags/3,tags/4,tags/5,tags/6,tags/7
0,YUMS x2,International food,NaN,NaN,NaN,NaN,NaN,NaN
1,International food,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Japanese,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Indian,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Portuguese,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1629,Portuguese,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1630,Italian,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1631,Portuguese,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1632,INSIDER,Steakhouse,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# df_tags['tags/0'].unique()

In [ ]:
# df_tags['tags/1'].unique()

In [ ]:
# df_tags['tags/2'].unique()

In [ ]:
# df_tags['tags/3'].unique()

In [ ]:
# df_tags['tags/4'].unique()

In [ ]:
# df_tags['tags/5'].unique()

In [ ]:
# df_tags['tags/6'].unique()

In [ ]:
# df_tags['tags/7'].unique()

In [ ]:
# df_tags.to_csv('tags.csv', index = True)

In [ ]:
df_tags['michelin'] = df_tags.apply(lambda row: 1 if 'MICHELIN' in row.values else 0, axis=1)

In [ ]:
df_tags[df_tags['michelin']==1]

,tags/0,tags/1,tags/2,tags/3,tags/4,tags/5,tags/6,tags/7,michelin
9,MICHELIN,INSIDER,Portuguese,NaN,NaN,NaN,NaN,NaN,1
101,MICHELIN,INSIDER,International food,Accepting my yums,NaN,NaN,NaN,NaN,1
130,MICHELIN,INSIDER,International food,NaN,NaN,NaN,NaN,NaN,1
157,MICHELIN,INSIDER,European,NaN,NaN,NaN,NaN,NaN,1
174,MICHELIN,INSIDER,Portuguese,NaN,NaN,NaN,NaN,NaN,1
246,MICHELIN,INSIDER,Portuguese,NaN,NaN,NaN,NaN,NaN,1
249,MICHELIN,INSIDER,European,NaN,NaN,NaN,NaN,NaN,1
262,MICHELIN,INSIDER,Italian,NaN,NaN,NaN,NaN,NaN,1
269,MICHELIN,Portuguese,NaN,NaN,NaN,NaN,NaN,NaN,1
286,MICHELIN,INSIDER,European,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
# #Removing TheFork based values
# df_tags = df_tags.replace('MICHELIN', np.nan)
# df_tags = df_tags.replace(['YUMS x2'], np.nan)
# df_tags = df_tags.replace(['Accepting my yums'], np.nan)
# df_tags = df_tags.replace(['INSIDER'], np.nan)

In [ ]:
# #unique values in the df_tags dataframe
# for col in df_tags.columns:
#     print(df_tags[col].unique())

[nan 'International food' 'Japanese' 'Indian' 'Portuguese' 'Pizzeria'
 'International' 'Mediterranean' 'Fusion' 'Nepalese' 'Italian' 'European'
 'Seafood' 'Vegan cuisine' 'Traditional cuisine' 'Steakhouse' 'Regional'
 'Greek' 'Vegetarian' 'Varied' 'Grilled' 'Thai' 'Mexican' 'Local' 'French'
 'Pub grub' 'Brazilian' 'Venezuelan' 'Peruvian' 'Korean' 'American'
 'Meat Cuisine' 'Asian' 'Toscano' 'African' 'Syrian' 'Iranian' 'Lebanese'
 'Tibetan' 'Vietnamese' 'Argentinian' 'Pugliese' 'Spanish']
['International food' nan 'Italian' 'Japanese' 'Portuguese' 'European'
 'Traditional cuisine' 'Welcome' 'Varied' 'International' 'Asian'
 'Mediterranean' 'Thai' 'Steakhouse' 'South American' 'Fusion' 'Pizzeria'
 'Meat Cuisine' 'Brazilian' 'Seafood' 'Regional' 'Grilled' 'Mexican'
 'Spanish' 'Indian' 'Peruvian' 'Cantonese' 'Vegan cuisine' 'Local'
 'French' 'Chinese']
[nan 'Portuguese' 'International food' 'Indian' 'European' 'Italian'
 'Mediterranean' 'Japanese' 'International' 'Vegetarian'
 'Traditiona

In [ ]:
# df_tags

,tags/0,tags/1,tags/2,tags/3,tags/4,tags/5,tags/6,tags/7,michelin
0,NaN,International food,NaN,NaN,NaN,NaN,NaN,NaN,0
1,International food,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,Japanese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Indian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Portuguese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
1629,Portuguese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1630,Italian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1631,Portuguese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1632,NaN,Steakhouse,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
# # remove the last 5 columns from the dataframe
# df_tags.drop(df_tags.columns[-6:], axis=1, inplace=True)

In [ ]:
# # join these columns in one column just with the non null values
# df_tags['type'] = df_tags.apply(lambda row: next((val for val in row if not pd.isna(val)), np.nan), axis=1)
# data[['type', 'michelin']] = df_tags[['type', 'michelin']].copy()

In [ ]:
# # delete all the columns except the new one
# df_tags.drop(df_tags.columns[:-1], axis=1, inplace=True)

In [ ]:
# # CREATE A CSV FILE WITH DF_TAGS DATAFRAME WITH NAME TYPES
# df_tags.to_csv('types.csv', index = True)

In [ ]:
# # compare type and cuisine columns 
# data['cuisine'].equals(data['type']) 

True

In [ ]:
data['michelin'] = df_tags['michelin'].copy()

#### Overall Dataframe

In [396]:
df_workable = data[['url', 'name', 'address', 'photo', 'averagePrice', 
                    'chefName1', 'chefName2', 'chefName3', 
                    'cuisine', 'michelin',
                    'description',  'isBookable', 'maxPartySize',
                    'schedule', 'promotions', 'phone', 'photo', 
                    'ratingValue', 'reviewCount', 'style',
                    'latitude', 'longitude', 'location', 'city', 
                    'ambienceRatingSummary', 'foodRatingSummary', 'serviceRatingSummary', 'paymentAcceptedSummary']].copy()

data['paymentsSummarystr'] = data['paymentAcceptedSummary'].apply(lambda x: str(x))
data['name_standard'] = data['name'].apply(lambda x: standardize_text(x))
data['paymentsSummarystr_standard'] = data['paymentsSummarystr'].apply(lambda x: standardize_text(x))
df_standards = data[['name_standard', 'cuisine_standard', 'style_standard', 'city_standard', 'location_standard', 'paymentsSummarystr_standard']].copy()


In [397]:
df_workable

,url,name,address,photo,averagePrice,chefName1,chefName2,chefName3,cuisine,description,isBookable,maxPartySize,schedule,promotions,phone,photo,ratingValue,reviewCount,style,latitude,longitude,location,city,ambienceRatingSummary,foodRatingSummary,serviceRatingSummary,paymentAcceptedSummary
restaurantID,,,,,,,,,,,,,,,,,,,,,,,,,,,
730060,https://www.thefork.com/restaurant/invictus-r7...,Invictus,"R. José Fontana 10,Albufeira, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,14,Not Applicable,Not Applicable,Not Applicable,International,NaN,True,60.0,"{'Monday': '12:00 - 15:00, 17:30 - 23:00', 'Tu...","[{'promotion_type': '20% off', 'day_of_week': ...",Not Available,https://res.cloudinary.com/tf-lab/image/upload...,8.8,28.0,Bachelor party,NaN,NaN,Albufeira,Albufeira,10.000000,10.000000,10.000000,"[MBWay, MBWay]"
805058,https://www.thefork.com/restaurant/galinha-da-...,Galinha da Vizinha - Albufeira,"R. Almeida Garrett 71,Albufeira, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,15,Not Applicable,Not Applicable,Not Applicable,International,NaN,True,NaN,"{'Monday': '12:00 - 22:30', 'Tuesday': '12:00 ...","[{'promotion_type': 'Free dessert', 'day_of_we...",965372680,https://res.cloudinary.com/tf-lab/image/upload...,9.3,301.0,After work,NaN,NaN,Albufeira,Albufeira,9.000000,9.100000,8.900000,"[MBWay, MBWay, Maestro Card, Mastercard]"
712669,https://www.thefork.com/restaurant/odessa-sush...,Odessa Sushi Restaurant,"R. da Torre Velha,Albufeira, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,24,Yana Malayko,Not Applicable,Not Applicable,Japanese,NaN,True,60.0,"{'Monday': 'Closed', 'Tuesday': '18:00 - 22:00...",No Offers,967781189,https://res.cloudinary.com/tf-lab/image/upload...,9.6,376.0,Birthday,NaN,NaN,Albufeira,Albufeira,9.800000,10.000000,9.500000,"[MBWay, MBWay, Visa]"
576521,https://www.thefork.com/restaurant/chutneys-r5...,Chutneys,"Tv. Antero de Quental 4B r/c,Albufeira, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,14,Iqbal Hossain Mohon,Not Applicable,Not Applicable,Indian,NaN,True,60.0,"{'Monday': '11:30 - 14:30, 18:00 - 24:00', 'Tu...","[{'promotion_type': '20% off', 'day_of_week': ...",289508393,https://res.cloudinary.com/tf-lab/image/upload...,9.1,580.0,Good for ceremonies,NaN,NaN,Albufeira,Albufeira,9.000000,9.700000,9.500000,[Credit Card]
802974,https://www.thefork.com/restaurant/audaces-r80...,Audaces,"R. Alexandre Herculano 11,Albufeira, Portugal",https://res.cloudinary.com/tf-lab/image/upload...,15,Romilson Santos,Not Applicable,Not Applicable,Portuguese,NaN,True,NaN,"{'Monday': 'Closed', 'Tuesday': '12:00 - 16:00...",No Offers,Not Available,https://res.cloudinary.com/tf-lab/image/upload...,9.0,82.0,After work,NaN,NaN,Albufeira,Albufeira,9.000000,9.750000,9.250000,"[MBWay, MBWay, Credit Card, Maestro Card, Mast..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501687,https://www.thefork.com/restaurant/combinacao-...,Combinaçao de Sabores,"Largo da Cruz Velha,Vila Nova de Famalicão, Po...",https://res.cloudinary.com/tf-lab/image/upload...,15,Not Applicable,Not Applicable,Not Applicable,Portuguese,NaN,True,24.0,"{'Monday': 'Closed', 'Tuesday': '12:00 - 14:30...","[{'promotion_type': '30% off', 'day_of_week': ...",912951884,https://res.cloudinary.com/tf-lab/image/upload...,8.7,88.0,After work,NaN,NaN,Vila Nova de Famalicão,Vila Nova de Famalicão,8.800000,9.400000,9.600000,"[American Express, Credit Card, Mastercard, Vi..."
724143,https://www.thefork.com/restaurant/il-gatto-gi...,Il Gatto Ginno,"Alameda Cónego Joaquim Fernandes 160,Vila Nova...",https://res.cloudinary.com/tf-lab/image/upload...,18,Gatto Ginno,Not Applicable,Not Applicable,Italian,NaN,True,60.0,"{'Monday': 'Closed', 'Tuesday': 'Closed', 'Wed...",No Offers,913390660,https://res.cloudinary.com/tf-lab/image/upload...,9.6,82.0,Contemporary cuisine,NaN,NaN,Vila Nova de Famalicão,Vila Nova de Famalicão,9.818182,9.636364,9.818182,[]
418275,https://ww

In [398]:
df_workable.to_csv('data/preprocessed_data.csv', index = True)
df_standards.to_csv('data/standards.csv', index = True)

### Restaurants Near You

In [399]:
df1 = df_workable[df_workable['location'] == 'Lisbon']
shaul = Location()
shaul.getLocation()

KeyboardInterrupt: 

In [ ]:
df1 = nearYou(shaul, df1, 10)

In [ ]:
df1['minutes_away'] = df1.apply(lambda row: shaul.getDirections(row['latitude'], row['longitude'], ['driving'])['driving'].minutes, axis=1)

In [ ]:
df1

,url,address,averagePrice,chefName,chefName2,chefName3,cuisine,currency,description,hasLoyaltyProgram,...,longitude,location,city,ambienceRatingSummary,foodRatingSummary,serviceRatingSummary,paymentAcceptedSummary,michelin,haversine_distances,minutes_away
740,https://www.thefork.com/restaurant/shun-open-k...,"R. Tomás da Anunciação 171A,Lisbon, Portugal",20,joao bivar,Not Applicable,Not Applicable,Japanese,EUR,NaN,False,...,-9.165850,Lisbon,Almada,9.333333,9.333333,8.666667,"[Mastercard, Credit Card, Visa Electron, Visa]",0,1.370685,8
811,https://www.thefork.com/restaurant/don-pablo-c...,"R. Silva Carvalho 167,Lisbon, Portugal",16,not applicable,Not Applicable,Not Applicable,Portuguese,EUR,NaN,False,...,-9.161786,Lisbon,Almada,9.200000,9.300000,9.300000,"[Visa Electron, Visa]",0,1.414799,8
812,https://www.thefork.com/restaurant/dali-cozinh...,"Rua de Infantaria 16, número 43, Campo de Ouri...",25,carol silva,Not Applicable,Not Applicable,International,EUR,NaN,False,...,-9.167584,Lisbon,Almada,9.300000,9.600000,9.300000,"[Credit Card, Visa Electron, Visa]",0,1.449274,6
1030,https://www.thefork.com/restaurant/obica-mozza...,"R. da Escola Politécnica 90,Lisbon, Portugal",25,luis minuzzi,Not Applicable,Not Applicable,Pugliese,EUR,NaN,False,...,-9.152127,Lisbon,Almada,8.900000,8.400000,8.600000,"[Mastercard, Apple Pay, Credit Card, Visa Elec...",0,1.697321,6
895,https://www.thefork.com/restaurant/santo-graal...,"R. Ferreira Borges 45,Lisbon, Portugal",13,vini lisboa,Not Applicable,Not Applicable,International food,EUR,NaN,False,...,-9.164038,Lisbon,Almada,9.400000,9.500000,9.500000,"[Visa Electron, Visa]",0,1.813681,10
846,https://www.thefork.com/restaurant/comida-de-s...,"Calçada Eng. Miguel Pais 39,Lisbon, Portugal",25,not applicable,Not Applicable,Not Applicable,Brazilian,EUR,NaN,False,...,-9.151491,Lisbon,Almada,9.600000,10.000000,10.000000,"[Mastercard, Visa, American Express]",0,1.823498,7
1139,https://www.thefork.com/restaurant/tasca-do-te...,"Rua das Taipas 41A,Lisbon, Portugal",25,mario vasconcelos,Not Applicable,Not Applicable,Portuguese,EUR,NaN,False,...,-9.145499,Lisbon,Almada,9.400000,9.500000,9.400000,[Credit Card],0,2.139528,11
767,https://www.thefork.com/restaurant/cafe-de-sao...,"Rua de São Bento 212,Lisbon, Portugal",45,not applicable,Not Applicable,Not Applicable,Portuguese,EUR,NaN,False,...,-9.153117,Lisbon,Almada,9.800000,9.500000,9.500000,"[Mastercard, Visa Electron, Visa, American Exp...",0,2.200824,8
922,https://www.thefork.com/restaurant/tamarind-r6...,"Rua da Glória 43,Lisbon, Portugal",15,not applicable,Not Applicable,Not Applicable,Indian,EUR,NaN,False,...,-9.144153,Lisbon,Almada,8.400000,9.400000,9.400000,"[Visa Electron, Visa]",0,2.203815,8
1036,https://www.thefork.com/restaurant/leve-leve-2...,"Tv. da Boa Hora 48,Lisbon, Portugal",15,not applicable,Not Applicable,Not Applicable,International,EUR,NaN,False,...,-9.145052,Lisbon,Almada,8.375000,9.000000,8.625000,"[Visa Electron, Visa]",0,2.422313,12


##  __Painel de Analytics (primeira página)__:
- dizer quantos restaurantes se encontraram perto (Fazer um plot);
- nome do restaurante mais próximo e distancia;
- qual o tipo de comida mais comum próximo;
- Qual o average price da comdia proximo;
- Qual o restaurante mais top rated e mesmo a nível de coisas especificas (ambiente, comida);
- Promoções perto.
- User poderá escolher a data para ver os restaurantes abertps;

In [ ]:
import folium

m = folium.Map(location=(shaul.latitude, shaul.longitude), zoom_start=7, tiles="cartodb positron")
for index, row in df1.iterrows():
    # Create a marker for each observation
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],  # Display the name in a popup
    ).add_to(m)

m

In [ ]:
df1['averagePrice'].median()

22.5

In [ ]:
df1['cuisine'].value_counts()

Portuguese            3
International         2
Japanese              1
Pugliese              1
International food    1
Brazilian             1
Indian                1
Name: cuisine, dtype: int64

In [ ]:
df1[df1['minutes_away'] == df1['minutes_away'].min()]

,url,address,averagePrice,chefName,chefName2,chefName3,cuisine,currency,description,hasLoyaltyProgram,...,longitude,location,city,ambienceRatingSummary,foodRatingSummary,serviceRatingSummary,paymentAcceptedSummary,michelin,haversine_distances,minutes_away
812,https://www.thefork.com/restaurant/dali-cozinh...,"Rua de Infantaria 16, número 43, Campo de Ouri...",25,carol silva,Not Applicable,Not Applicable,International,EUR,NaN,False,...,-9.167584,Lisbon,Almada,9.3,9.6,9.3,"[Credit Card, Visa Electron, Visa]",0,1.449274,6
1030,https://www.thefork.com/restaurant/obica-mozza...,"R. da Escola Politécnica 90,Lisbon, Portugal",25,luis minuzzi,Not Applicable,Not Applicable,Pugliese,EUR,NaN,False,...,-9.152127,Lisbon,Almada,8.9,8.4,8.6,"[Mastercard, Apple Pay, Credit Card, Visa Elec...",0,1.697321,6


In [ ]:
df1[df1['ratingValue'] == df1['ratingValue'].max()]

,url,address,averagePrice,chefName,chefName2,chefName3,cuisine,currency,description,hasLoyaltyProgram,...,longitude,location,city,ambienceRatingSummary,foodRatingSummary,serviceRatingSummary,paymentAcceptedSummary,michelin,haversine_distances,minutes_away
1139,https://www.thefork.com/restaurant/tasca-do-te...,"Rua das Taipas 41A,Lisbon, Portugal",25,mario vasconcelos,Not Applicable,Not Applicable,Portuguese,EUR,NaN,False,...,-9.145499,Lisbon,Almada,9.4,9.5,9.4,[Credit Card],0,2.139528,11
767,https://www.thefork.com/restaurant/cafe-de-sao...,"Rua de São Bento 212,Lisbon, Portugal",45,not applicable,Not Applicable,Not Applicable,Portuguese,EUR,NaN,False,...,-9.153117,Lisbon,Almada,9.8,9.5,9.5,"[Mastercard, Visa Electron, Visa, American Exp...",0,2.200824,8


In [ ]:
df1[df1['foodRatingSummary'] == df1['foodRatingSummary'].max()]

,url,address,averagePrice,chefName,chefName2,chefName3,cuisine,currency,description,hasLoyaltyProgram,...,longitude,location,city,ambienceRatingSummary,foodRatingSummary,serviceRatingSummary,paymentAcceptedSummary,michelin,haversine_distances,minutes_away
846,https://www.thefork.com/restaurant/comida-de-s...,"Calçada Eng. Miguel Pais 39,Lisbon, Portugal",25,not applicable,Not Applicable,Not Applicable,Brazilian,EUR,NaN,False,...,-9.151491,Lisbon,Almada,9.6,10.0,10.0,"[Mastercard, Visa, American Express]",0,1.823498,7


In [ ]:
df1[df1['ambienceRatingSummary'] == df1['ambienceRatingSummary'].max()]

,url,address,averagePrice,chefName,chefName2,chefName3,cuisine,currency,description,hasLoyaltyProgram,...,longitude,location,city,ambienceRatingSummary,foodRatingSummary,serviceRatingSummary,paymentAcceptedSummary,michelin,haversine_distances,minutes_away
767,https://www.thefork.com/restaurant/cafe-de-sao...,"Rua de São Bento 212,Lisbon, Portugal",45,not applicable,Not Applicable,Not Applicable,Portuguese,EUR,NaN,False,...,-9.153117,Lisbon,Almada,9.8,9.5,9.5,"[Mastercard, Visa Electron, Visa, American Exp...",0,2.200824,8


In [ ]:
df1[df1['serviceRatingSummary'] == df1['serviceRatingSummary'].max()]

,url,address,averagePrice,chefName,chefName2,chefName3,cuisine,currency,description,hasLoyaltyProgram,...,longitude,location,city,ambienceRatingSummary,foodRatingSummary,serviceRatingSummary,paymentAcceptedSummary,michelin,haversine_distances,minutes_away
846,https://www.thefork.com/restaurant/comida-de-s...,"Calçada Eng. Miguel Pais 39,Lisbon, Portugal",25,not applicable,Not Applicable,Not Applicable,Brazilian,EUR,NaN,False,...,-9.151491,Lisbon,Almada,9.6,10.0,10.0,"[Mastercard, Visa, American Express]",0,1.823498,7


In [ ]:
df1[df1['promotions'] != 'No Offers']

,url,address,averagePrice,chefName,chefName2,chefName3,cuisine,currency,description,hasLoyaltyProgram,...,longitude,location,city,ambienceRatingSummary,foodRatingSummary,serviceRatingSummary,paymentAcceptedSummary,michelin,haversine_distances,minutes_away
740,https://www.thefork.com/restaurant/shun-open-k...,"R. Tomás da Anunciação 171A,Lisbon, Portugal",20,joao bivar,Not Applicable,Not Applicable,Japanese,EUR,NaN,False,...,-9.165850,Lisbon,Almada,9.333333,9.333333,8.666667,"[Mastercard, Credit Card, Visa Electron, Visa]",0,1.370685,8
895,https://www.thefork.com/restaurant/santo-graal...,"R. Ferreira Borges 45,Lisbon, Portugal",13,vini lisboa,Not Applicable,Not Applicable,International food,EUR,NaN,False,...,-9.164038,Lisbon,Almada,9.400000,9.500000,9.500000,"[Visa Electron, Visa]",0,1.813681,10
1139,https://www.thefork.com/restaurant/tasca-do-te...,"Rua das Taipas 41A,Lisbon, Portugal",25,mario vasconcelos,Not Applicable,Not Applicable,Portuguese,EUR,NaN,False,...,-9.145499,Lisbon,Almada,9.400000,9.500000,9.400000,[Credit Card],0,2.139528,11


# Menu Integration

In [400]:
menu_df = pd.read_csv('data/almost_clean_menus.csv')

In [401]:
df_workable = df_workable.merge(menu_df, left_on = 'url', right_on='input', how='left')

In [ ]:
# df_workable.drop(['input'], axis=1, inplace=True)

In [ ]:
# df_workable.to_csv('data/df_workable.csv', index=False)

In [402]:
df_workable = pd.read_csv('data/preprocessed_data.csv')

In [ ]:
df_workable

,url,name,address,averagePrice,chefName1,chefName2,chefName3,cuisine,description,isBookable,maxPartySize,schedule,promotions,phone,photo,ratingValue,reviewCount,style,latitude,longitude,location,city,ambienceRatingSummary,foodRatingSummary,serviceRatingSummary,paymentAcceptedSummary,input,menu_pre_proc
0,https://www.thefork.com/restaurant/invictus-r7...,Invictus,"R. José Fontana 10,Albufeira, Portugal",14,Not Applicable,Not Applicable,Not Applicable,International,NaN,True,60.0,"{'Monday': '12:00 - 15:00, 17:30 - 23:00', 'Tu...","[{'promotion_type': '20% off', 'day_of_week': ...",Not Available,https://res.cloudinary.com/tf-lab/image/upload...,8.8,28.0,Bachelor party,NaN,NaN,Albufeira,Albufeira,10.000000,10.000000,10.000000,"[MBWay, MBWay]",https://www.thefork.com/restaurant/invictus-r7...,{'Carnes ': {'Picanha fatiada': {'isMainDish':...
1,https://www.thefork.com/restaurant/galinha-da-...,Galinha da Vizinha - Albufeira,"R. Almeida Garrett 71,Albufeira, Portugal",15,Not Applicable,Not Applicable,Not Applicable,International,NaN,True,NaN,"{'Monday': '12:00 - 22:30', 'Tuesday': '12:00 ...","[{'promotion_type': 'Free dessert', 'day_of_we...",965372680,https://res.cloudinary.com/tf-lab/image/upload...,9.3,301.0,After work,NaN,NaN,Albufeira,Albufeira,9.000000,9.100000,8.900000,"[MBWay, MBWay, Maestro Card, Mastercard]",https://www.thefork.com/restaurant/galinha-da-...,{'Para começar': {'Cestinho de pão da vizinha'...
2,https://www.thefork.com/restaurant/odessa-sush...,Odessa Sushi Restaurant,"R. da Torre Velha,Albufeira, Portugal",24,Yana Malayko,Not Applicable,Not Applicable,Japanese,NaN,True,60.0,"{'Monday': 'Closed', 'Tuesday': '18:00 - 22:00...",No Offers,967781189,https://res.cloudinary.com/tf-lab/image/upload...,9.6,376.0,Birthday,NaN,NaN,Albufeira,Albufeira,9.800000,10.000000,9.500000,"[MBWay, MBWay, Visa]",https://www.thefork.com/restaurant/odessa-sush...,{'Entradas': {'Edamame picante': {'isMainDish'...
3,https://www.thefork.com/restaurant/chutneys-r5...,Chutneys,"Tv. Antero de Quental 4B r/c,Albufeira, Portugal",14,Iqbal Hossain Mohon,Not Applicable,Not Applicable,Indian,NaN,True,60.0,"{'Monday': '11:30 - 14:30, 18:00 - 24:00', 'Tu...","[{'promotion_type': '20% off', 'day_of_week': ...",289508393,https://res.cloudinary.com/tf-lab/image/upload...,9.1,580.0,Good for ceremonies,NaN,NaN,Albufeira,Albufeira,9.000000,9.700000,9.500000,[Credit Card],https://www.thefork.com/restaurant/chutneys-r5...,"{'Entradas': {'Papri': {'isMainDish': False, '..."
4,https://www.thefork.com/restaurant/audaces-r80...,Audaces,"R. Alexandre Herculano 11,Albufeira, Portugal",15,Romilson Santos,Not Applicable,Not Applicable,Portuguese,NaN,True,NaN,"{'Monday': 'Closed', 'Tuesday': '12:00 - 16:00...",No Offers,Not Available,https://res.cloudinary.com/tf-lab/image/upload...,9.0,82.0,After work,NaN,NaN,Albufeira,Albufeira,9.000000,9.750000,9.250000,"[MBWay, MBWay, Credit Card, Maestro Card, Mast...",https://www.thefork.com/restaurant/audaces-r80...,{'Entradas': {'Creme de legumes e crôton de pã...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1517,https://www.thefork.com/restaurant/combinacao-...,Combinaçao de Sabores,"Largo da Cruz Velha,Vila Nova de Famalicão, Po...",15,Not Applicable,Not Applicable,Not Applicable,Portuguese,NaN,True,24.0,"{'Monday': 'Closed', 'Tuesday': '12:00 - 14:30...","[{'promotion_type': '30% off', 'day_of_week': ...",912951884,https://res.cloudinary.com/tf-lab/image/upload...,8.7,88.0,After work,NaN,NaN,Vila Nova de Famalicão,Vila Nova de Famalicão,8.800000,9.400000,9.600000,"[American Express, Credit Card, Mastercard, Vi...",https://www.thefork.com/restaurant/combinacao-...,{'Especialidades': {'Francesinha': {'isMainDis...
1518,https://www.thefork.com/restaurant/il-gatto-gi...,Il Gatto Ginno,"Alameda Cónego Joaquim Fernandes 160,Vila Nova...",18,Gatto Ginno,Not Applicable,Not Applicable,Italian,NaN,True,60.0,"{'Monday': 'Closed', 'Tuesday': 'Closed', 'Wed...",